In [ ]:
import pandas as pd

In [ ]:
data_path = '/kaggle/input/cat-in-the-dat-ii/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [ ]:
train.shape, test.shape

In [ ]:
train.head().T

In [ ]:
test.head()

In [ ]:
submission.head()

### 피처 요약표 만들기

In [ ]:
def resumetable(df):
    print(f'데이터셋 형상: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': '피처'})
    summary['결측값 개수'] = df.isnull().sum().values
    summary['고윳값 개수'] = df.nunique().values
    summary['첫 번째 값'] = df.loc[0].values
    summary['두 번째 값'] = df.loc[1].values
    summary['세 번째 값'] = df.loc[2].values
    
    return summary

In [ ]:
feature_resume_table = resumetable(train)
feature_resume_table

### 피처 요약표 해석하기

1. 이진(binary) 피처 : bin_0 ~ bin_4
2. 명목형(nominal) 피처 : nom_0 ~ nom_9
3. 순서형(ordinal) 피처 : ord_0 ~ ord_5
4. 그 외 피처 : day, month, target

In [ ]:
feature_resume_table[feature_resume_table['피처'].str.startswith('bin')]

이진 피처

고윳값이 모두 2개

bin_0 ~ bin_2 는 데이터 타입이 float64  실제값이 0.0, 1.0 으로 구성

bin_3, bin_4 는 object 타입이고 실제값은 T 또는 F (bin_3), Y 또는 N (bin_4)

결측값은 약 18000개씩 존재 

- 최빈값으로 처리할까?

In [ ]:
feature_resume_table[feature_resume_table['피처'].str.startswith('nom')]

명목형 피처

모두 object 타입이고 결측값이 약 18000개 정도씩 있음

nom_0 ~ nom_4 는 고윳값이 6개 이하인데, nom_5 ~ nom_9 는 고윳값이 많음

nom_5 ~ nom_9 는 알 수 없는 값이 입력되어 있음

In [ ]:
feature_resume_table[feature_resume_table['피처'].str.startswith('ord')]

###### 순서형 피처

ord_0 피처만 float64 타입이고 나머지는 object 타입

결측값은 역시 약 18000개

순서를 파악하기 위해 순서형 피처의 고윳값을 출력해보자

고윳값 개수가 적은 ord_0 ~ ord_5 피처부터 보자

In [ ]:
for i in range(6):
    feature = 'ord_' + str(i)
    print(f'{feature} 고윳값: {train[feature].unique()}')

ord_0 는 숫자로 되어있어서 숫자 크기에 순서를 맞춤

ord_1 은 'Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster' 순으로 맞춤

ord_2 는 'Freezing', 'Cold', 'Warm', 'Hot', 'Boiling Hot', 'Lava Hot' 순으로 맞춤

ord_3 ~ ord_5 는 알파벳 순으로 맞춤

In [ ]:
feature_resume_table[~feature_resume_table['피처'].str.contains('_')]

In [ ]:
print('day 고윳값:', train['day'].unique())
print('month 고윳값:', train['month'].unique())
print('target 고윳값:', train['target'].unique())

day 의 고윳값은 7개 이므로 요일을 나타낸다고 짐작할 수 있음

month 의 고윳값은 1 ~ 12 이므로 월을 나타냄

target 의 고윳값은 0 또는 1로 구성

## 데이터 시각화

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

### 타깃값 분포

In [ ]:
def write_percent(ax, total_size):
    '''도형 객체를 순회하며 막대 상단에 타깃값 비율 표시'''
    for patch in ax.patches:
        height = patch.get_height()
        width = patch.get_width()
        left_coord = patch.get_x()
        percent = height / total_size * 100
        
        ax.text(x=left_coord + width / 2.0,
                y=height + total_size * 0.001,
                s=f'{percent:1.1f}%',
                ha='center')

In [ ]:
mpl.rc('font', size=15)
plt.figure(figsize=(7, 6))

# 타깃값 분포 카운트플롯
ax = sns.countplot(x='target', data=train)
write_percent(ax, len(train))
ax.set_title('Target Distribution')

- 분석 결과 : 타깃값 0과 1이 약 8대 2 비율

### 이진 피처 분포

In [ ]:
import matplotlib.gridspec as gridspec

In [ ]:
# 3행 2열 틀(Figure) 준비
mpl.rc('font', size=12)
grid = gridspec.GridSpec(3, 2)
plt.figure(figsize=(10, 16))
plt.subplots_adjust(wspace=0.4, hspace=0.3)

# 서브플롯 그리기
bin_features = ['bin_' + str(i) for i in range(5)]

for idx, feature in enumerate(bin_features):
    ax = plt.subplot(grid[idx])
    
    # ax 축에 타깃값 분포 카운트플롯 그리기
    sns.countplot(x=feature,
                  data=train,
                  hue='target',
                  palette='pastel',
                  ax=ax)
    
    ax.set_title(f'{feature} Distribution by Target')
    write_percent(ax, len(train))

- 분석 결과 : 고윳값별로 나눠봐도 타깃값 0, 1의 분포가 대체포 4:1 수준임, 특정 타깃값에 치우치지 않았음을 확인할 수 있음

### 명목형 피처 분포

nom_5 ~ nom_9 피처는 고윳값 개수가 많고 의미를 알 수 없는 문자열이 입력되어 있으니 nom_0 ~ nom_4 피처까지만 시각화하기로 함

In [ ]:
def get_crosstab(df, feature):
    crosstab = pd.crosstab(df[feature], df['target'], normalize='index') * 100
    crosstab = crosstab.reset_index()
    return crosstab

In [ ]:
def plot_pointplot(ax, feature, crosstab):
    ax2 = ax.twinx()
    
    ax2 = sns.pointplot(x=feature, y=1, data=crosstab,
                        order=crosstab[feature].values,
                        color='black',
                        legend=False,
                       )
    ax2.set_ylim(crosstab[1].min() - 5, crosstab[1].max() * 1.1)
    ax2.set_ylabel('Target 1 Ratio(%)')

In [ ]:
def plot_cat_dist_with_true_ratio(df, features, num_rows, num_cols, size=(15, 20)):
    plt.figure(figsize=size)
    grid = gridspec.GridSpec(num_rows, num_cols)
    plt.subplots_adjust(wspace=0.4, hspace=0.3)
    
    for idx, feature in enumerate(features):
        ax = plt.subplot(grid[idx])
        crosstab = get_crosstab(df, feature)
        
        sns.countplot(x=feature, data=df,
                      order=crosstab[feature].values,
                      color='skyblue',
                      ax=ax)
        
        write_percent(ax, len(df))
        
        plot_pointplot(ax, feature, crosstab)
        
        ax.set_title(f'{feature} Distribution')

In [ ]:
nom_features = ['nom_' + str(i) for i in range(5)]
plot_cat_dist_with_true_ratio(train, nom_features, num_rows=3, num_cols=2)

- 분석 결과 : nom_0 ~ nom_4 피처는 고윳값별로 타깃값 1 비율이 서로 다름, 고윳값 개수도 적으니 원-핫 인코딩 적용

### 순서형 피처 분포

In [ ]:
from pandas.api.types import CategoricalDtype

In [ ]:
ord_1_value = ['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster']
ord_2_value = ['Freezing', 'Cold', 'Warm', 'Hot', 'Boiling Hot', 'Lava Hot']

# 순서를 지정한 범주형 데이터 타입
ord_1_dtype = CategoricalDtype(categories=ord_1_value, ordered=True)
ord_2_dtype = CategoricalDtype(categories=ord_2_value, ordered=True)

# 데이터 타입 변경
train['ord_1'] = train['ord_1'].astype(ord_1_dtype)
train['ord_2'] = train['ord_2'].astype(ord_2_dtype)

In [ ]:
ord_features = ['ord_' + str(i) for i in range(4)]
plot_cat_dist_with_true_ratio(train, ord_features, num_rows=2, num_cols=2, size=(15, 12))

- 분석 결과 : ord_0 ~ ord_3 모두 고윳값 순서에 따라 타깃값 1 비율도 비례해서 커짐

In [ ]:
plot_cat_dist_with_true_ratio(train, ['ord_4', 'ord_5'], num_rows=2, num_cols=1, size=(15, 12))

- 분석 결과 : ord_4, ord_5 모두 고윳값 순서에 따라 타깃값 1 비율이 증가

### 날짜 피처 분포

In [ ]:
date_features = ['day', 'month']
plot_cat_dist_with_true_ratio(train, date_features, num_rows=2, num_cols=1, size=(10, 10))

- 분석 결과 : 
    - day 피처는 1에서 4로 갈수록 타깃값 1 비율이 줄어들고 4에서 7로 갈수록 비율이 증가
    - month 피처는 1에서 2, 6에서 8, 11에서 12로 갈때 줄어들고 2에서 6, 8에서 11로 갈 때 비율이 증가